In [1]:
import pandas as pd

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df

,Lugar de residencia,Medida,Valor
0,Huelva,Población,510743
1,Huelva,Confirmados PDIA,8784
2,Huelva,Confirmados PDIA 14 días,2542
3,Huelva,Tasa PDIA 14 días,"497,706282807596"
4,Huelva,Confirmados PDIA 7 días,1087
...,...,...,...
627,Municipio de Huelva sin especificar,Tasa PDIA 14 días,-
628,Municipio de Huelva sin especificar,Confirmados PDIA 7 días,5
629,Municipio de Huelva sin especificar,Total Confirmados,108
630,Municipio de Huelva sin especificar,Curados,42


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  8784.0


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  2456.0


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 410 personas en los últimos 7 días 

Un positivo PCR cada 193 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,8784.0,1087.0,2542.0,510743.0,212.827195,497.706283,411.0
Huelva-Costa,4748.0,679.0,1521.0,288115.0,235.669785,527.914201,277.0
Huelva (capital),2456.0,350.0,744.0,143663.0,243.625707,517.878647,137.0
Condado-Campiña,2744.0,325.0,762.0,155057.0,209.600341,491.432183,109.0
Lepe,753.0,119.0,262.0,27431.0,433.815756,955.123765,63.0
Moguer,247.0,48.0,118.0,22088.0,217.312568,534.226729,23.0
Sierra de Huelva-Andévalo Central,1185.0,78.0,251.0,67571.0,115.434136,371.461130,21.0
Aljaraque,319.0,31.0,85.0,21260.0,145.813735,399.811853,17.0
Palma del Condado (La),397.0,43.0,114.0,10761.0,399.591116,1059.381098,16.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() > 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Corteconcepción,17.0,11.0,14.0,548.0,2007.299270,2554.744526,2.0
Palma del Condado (La),397.0,43.0,114.0,10761.0,399.591116,1059.381098,16.0
Lepe,753.0,119.0,262.0,27431.0,433.815756,955.123765,63.0
Villablanca,54.0,5.0,27.0,2848.0,175.561798,948.033708,2.0
Lucena del Puerto,54.0,14.0,30.0,3371.0,415.307031,889.943637,5.0
Villarrasa,88.0,3.0,19.0,2176.0,137.867647,873.161765,1.0
Minas de Riotinto,58.0,12.0,28.0,3848.0,311.850312,727.650728,3.0
Palos de la Frontera,165.0,25.0,79.0,11289.0,221.454513,699.796262,13.0
Niebla,88.0,9.0,25.0,4117.0,218.605781,607.238280,3.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Paterna del Campo,50.0,1.0,7.0,3509.0,28.498148,199.487033,1.0,0.142857
Aracena,278.0,7.0,45.0,8107.0,86.345134,555.075860,0.0,0.155556
Villarrasa,88.0,3.0,19.0,2176.0,137.867647,873.161765,1.0,0.157895
Campillo (El),51.0,2.0,12.0,2023.0,98.863075,593.178448,0.0,0.166667
Jabugo,34.0,2.0,12.0,2250.0,88.888889,533.333333,0.0,0.166667
Villablanca,54.0,5.0,27.0,2848.0,175.561798,948.033708,2.0,0.185185
San Juan del Puerto,131.0,5.0,27.0,9300.0,53.763441,290.322581,1.0,0.185185
Valverde del Camino,257.0,13.0,68.0,12820.0,101.404056,530.421217,5.0,0.191176
Nerva,74.0,4.0,16.0,5235.0,76.408787,305.635148,1.0,0.250000
